In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# My libraries
import sys
sys.path.append('../') 

import models
import utils

In [2]:
numberOfTimeStep = 14
folders = ["s1", "s2", "s3"]

norm = "robustNorm"

device = torch.device('cuda:1')
print("Seleccionando la primera GPU:", torch.cuda.get_device_name(device))

Seleccionando la primera GPU: NVIDIA GeForce RTX 3090


## Hyperparameteres

In [3]:
# Always the same values, do not touch. 
in_dim_GCN = 1
out_dim_GCN = 1

# number of max epochs 
n_epochs = 1000

# Early stopping configuration
early_stopping_patience = 50

# Hyperparameters to be optimized (change this values)
h_dropout = [0.0, 0.15, 0.3]
h_learning_rate = [1e-3, 1e-2, 5e-2, 0.1]
h_decay = [0, 1e-5, 1e-4, 1e-3, 1e-2]
h_hid_lay = [4, 8, 16, 32, 64]
h_layers = [1, 2, 3, 4, 5, 6]
seed = [42, 76, 124, 163, 192, 205, 221, 245, 293]

fc_layer = [[1120, out_dim_GCN]]

# Parameters to define type of GCNN and type of output.
typeGCN = "standard_gcnn"
K = [0]

params = {# Hyperparameters
         'h_layers':h_layers, 'n_epochs':n_epochs, 
          'h_dropout': h_dropout, 'h_learning_rate': h_learning_rate, 
          'h_decay':h_decay, 'h_hid_lay': h_hid_lay, 'K':K,
          'fc_layer': fc_layer,
          # seed to set initialization hyperparameters
          'seed': seed, 
          # Type of output GCN
          'typeGCN': typeGCN,
          # Dimensions of GCN (input/output)
          'in_dim_GCN': in_dim_GCN, 'out_dim_GCN': out_dim_GCN,
          # Patiente
          'early_stopping_patience':early_stopping_patience}

## Way to build the network: 
### Product Graph of the same adjacency matrix obtained by dtw

In [4]:
way_to_build_graph = "dtw-hgd"
# Others ways: correlations; smoothness

## Train phase

In [ ]:
best_result_by_split = {}

for carp in range(len(folders)):
    torch.cuda.empty_cache()
    # Load data
    X_train_vec, X_val_vec, X_test_vec, y_train, y_val, y_test = utils.load_data(norm, device, folders[carp])
    A = pd.read_csv("../../"+way_to_build_graph+"/"+folders[carp]+"/SpaceTimeGraph_Xtr_"+norm+"_th_0.975.csv")
    A = torch.tensor(np.array(A), dtype=torch.float32)

    print("===========> TRAIN-VAL PHASE ==================")
    bestHyperparameters = utils.train_val_phase(A, X_train_vec, X_val_vec, y_train, y_val, params, device)
    print("<========== END TRAIN-VAL PHASE ===============")
    best_result_by_split[folders[carp]] = bestHyperparameters 
    
utils.saveBestHyperparameters(best_result_by_split, "../hyperparameters/"+way_to_build_graph+"/#E3.1-SpaceTimeGraph_th_0.975.json")
torch.cuda.empty_cache()

## Validation phase

In [5]:
folders = ["s1", "s2", "s3"]
best_result_by_split = utils.loadBestHyperparameters("../hyperparameters/"+way_to_build_graph+"/#E3.1-SpaceTimeGraph_th_0.975.json")

typeOfGraph = "SpaceTimeGraph"
path_A = typeOfGraph+"_Xtr_"+norm+"_th_0.975.csv"
results, importance_nodes, fc_classifiers, gnn_models = utils.val_model(best_result_by_split, typeOfGraph, params, folders, norm, device, path_A, way_to_build_graph)

keys = list(results.keys())
for c in range(len(folders)):
    print("================= SPLIT " + str(folders[c]) + " ===================")
    print(keys[1] + ": " + str(np.round(results[keys[1]][c]*100,2)))
    print(keys[2] + ": " + str(np.round(results[keys[2]][c]*100,2)))
    print(keys[3] + ": " + str(np.round(results[keys[3]][c]*100,2)))
    
print()
str_result = ""
for i in range(len(results.keys())):
    average = np.mean(results[keys[i]])
    std = np.std(results[keys[i]])
    print(keys[i] + ": " + str(np.round(average*100,2)) + " +- " + str(np.round(std*100, 2)))
    if i in [1,2]:
        str_result += str(np.round(average*100,2)) + " +- " + str(np.round(std*100, 2)) + " & "
    elif i in [3]:
        str_result += str(np.round(average*100,2)) + " +- " + str(np.round(std*100, 2)) + " \\\ "
print("\n",str_result)

torch.cuda.empty_cache()

Early stopping at epoch 290
Early stopping at epoch 153
Early stopping at epoch 116
================= SPLIT s1 ===================
roc_auc: 77.36
sensitivity: 75.47
specificity: 70.71
================= SPLIT s2 ===================
roc_auc: 79.65
sensitivity: 81.13
specificity: 70.71
================= SPLIT s3 ===================
roc_auc: 77.52
sensitivity: 71.7
specificity: 75.42

test_acc: 72.86 +- 1.46
roc_auc: 78.17 +- 1.04
sensitivity: 76.1 +- 3.88
specificity: 72.28 +- 2.22

 78.17 +- 1.04 & 76.1 +- 3.88 & 72.28 +- 2.22 \\ 


In [ ]:
 78.17 +- 1.04 & 76.1 +- 3.88 & 72.28 +- 2.22 \\ 